In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16982, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 16982 (delta 90), reused 110 (delta 54), pack-reused 16805 (from 1)
Receiving objects: 100% (16982/16982), 15.71 MiB | 24.26 MiB/s, done.
Resolving deltas: 100% (11628/11628), done.


In [2]:
%cd yolov5/
!pip install -r requirements.txt


/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.7/882.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


In [3]:
import torch
import cv2
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load pre-trained YOLOv5 model (use YOLOv5s, YOLOv5m, YOLOv5l, or YOLOv5x as needed)
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

# Path to the directories containing the images
image_folders = ['/content/drive/MyDrive/yolo images/Chemed', '/content/drive/MyDrive/yolo images/lobelia4cosmetics']

# Create a directory in your Google Drive to store the results
results_base_dir = '/content/drive/MyDrive/yolo_detection_results'
os.makedirs(results_base_dir, exist_ok=True)

for image_folder in image_folders:
    folder_name = os.path.basename(image_folder)

    # Create a list to store all detections
    all_detections = []

    # Process results to extract bounding box, class labels, and confidence scores
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)

        # Check if it's a valid image file (optional)
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # Run object detection
            results = model(img_path)

            # Extract data from results
            detections = results.pandas().xyxy[0]  # Pandas dataframe of detection results
            detections['image_name'] = img_name  # Add image name to the detections

            # Append detections to the list
            all_detections.append(detections[['image_name', 'name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']])

    # Combine all detections into a single DataFrame
    if all_detections:
        combined_detections = pd.concat(all_detections, ignore_index=True)

        # Save combined detections to a single CSV file
        csv_filename = f'{folder_name}_all_detections.csv'
        csv_path = os.path.join(results_base_dir, csv_filename)
        combined_detections.to_csv(csv_path, index=False)

print("All images processed. Results saved in:", results_base_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-12 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


All images processed. Results saved in: /content/drive/MyDrive/yolo_detection_results
